In [5]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [6]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import sys
import argparse
import subprocess
import pdb
import time
import random
import _pickle as cPickle
import matplotlib.pyplot as plt
import glob

%matplotlib inline

import numpy as np
import pandas as pd
import tensorflow as tf

from data_structure import get_batches
from hntm import HierarchicalNeuralTopicModel
from tree import get_descendant_idxs
from evaluation import validate, get_hierarchical_affinity, get_topic_specialization, print_topic_sample
from configure import get_config

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# load data & set config

In [7]:
config = get_config(nb_name)

In [8]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu
np.random.seed(config.seed)
random.seed(config.seed)

In [9]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_batches(instances_test, config.batch_size)
config.dim_bow = len(bow_idxs)

In [10]:
def debug_shape(variables, model):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)

def debug_value(variables, model, return_value=False):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)

    return _variables

# run

## initialize log

In [13]:
checkpoint = []
losses_train = []
ppls_train = []
ppl_min = np.inf
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','','','','','VALID:','','','','','TEST:','', 'SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL','REG','LOSS','PPL','NLL','KL','REG','LOSS','PPL', '1', '2', '3', 'CHILD', 'OTHER']]))))

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

def update_checkpoint(config, checkpoint, global_step):
    checkpoint.append(config.path_model + '-%i' % global_step)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0) + '.*'
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize model

In [14]:
if 'sess' in globals(): sess.close()
model = HierarchicalNeuralTopicModel(config)
sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(max_to_keep=config.max_to_keep)
update_tree_flg = False

## train & validate model

In [15]:
time_start = time.time()
while epoch < config.n_epochs:
    # train
    for ct, batch in train_batches:
        feed_dict = model.get_feed_dict(batch)
        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, ppls_batch, global_step_log = \
        sess.run([model.opt, model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_reg, model.topic_ppls, tf.train.get_global_step()], feed_dict = feed_dict)

        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch]]
        ppls_train += list(ppls_batch)

        if global_step_log % config.log_period == 0:
            # validate
            loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train = np.mean(losses_train, 0)
            ppl_train = np.exp(np.mean(ppls_train))
            loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)

            # test
            if ppl_dev < ppl_min:
                ppl_min = ppl_dev
                loss_test, _, _, _, ppl_test, _ = validate(sess, test_batches, model)
                saver.save(sess, config.path_model, global_step=global_step_log)
                cPickle.dump(config, open(config.path_config % global_step_log, 'wb'))
                update_checkpoint(config, checkpoint, global_step_log)
            
            # visualize topic
            topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
            topics_freq_idxs = bow_idxs[topics_freq_indices]
            topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
            topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
            descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
            recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
            
            depth_specs = get_topic_specialization(sess, model, instances_test)
            hierarchical_affinities = get_hierarchical_affinity(sess, model)
            
            # log
            clear_output()
            time_log = int(time.time() - time_start)
            log_series = pd.Series([time_log, epoch, ct, \
                    '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_reg_train, \
                    '%.2f'%loss_dev, '%.0f'%ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_reg_dev, \
                    '%.2f'%loss_test, '%.0f'%ppl_test, \
                    '%.2f'%depth_specs[1], '%.2f'%depth_specs[2], '%.2f'%depth_specs[3], \
                    '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
                    index=log_df.columns)
            log_df.loc[global_step_log] = log_series
            display(log_df)
            cPickle.dump(log_df, open(os.path.join(config.path_log), 'wb'))
            print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

            # update tree
            if not config.static:
                config.tree_idxs, update_tree_flg = model.update_tree(topic_prob_topic, recur_prob_topic)
                if update_tree_flg:
                    print(config.tree_idxs)
                    name_variables = {tensor.name: variable for tensor, variable in zip(tf.global_variables(), sess.run(tf.global_variables()))} # store paremeters
                    if 'sess' in globals(): sess.close()
                    model = HierarchicalNeuralTopicModel(config)
                    sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
                    name_tensors = {tensor.name: tensor for tensor in tf.global_variables()}
                    sess.run([name_tensors[name].assign(variable) for name, variable in name_variables.items()]) # restore parameters
                    saver = tf.train.Saver(max_to_keep=1)
                
            time_start = time.time()

    train_batches = get_batches(instances_train, config.batch_size, iterator=True)
    epoch += 1

loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)
topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
topics_freq_idxs = bow_idxs[topics_freq_indices]
topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
display(log_df)
print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

TRAIN:                           VALID:               \
       Time   Ep   Ct    LOSS  PPL     NLL    KL   REG    LOSS  PPL     NLL   
5000    192   10    9  111.45  491  110.45  1.00  0.01  103.91  459  102.65   
10000   217   20   19  111.05  471  109.75  1.28  0.01  103.68  442  102.08   
15000   215   30   29  110.81  459  109.31  1.49  0.02  103.61  435  101.81   
20000   208   40   39  110.63  451  108.99  1.63  0.02  103.49  428  101.53   
25000   239   50   49  110.51  444  108.74  1.75  0.02  103.39  421  101.29   
30000   262   60   59  110.40  439  108.55  1.84  0.02  103.33  417  101.17   
35000   225   70   69  110.32  435  108.39  1.92  0.02  103.31  415  101.07   
40000   273   80   79  110.25  432  108.25  1.98  0.01  103.24  413  100.98   
45000   292   90   89  110.19  429  108.14  2.03  0.01  103.22  410  100.92   
50000   202  100   99  110.13  426  108.04  2.08  0.01  103.20  409  100.83   
55000   382  110  109  110.09  424  107.95  2.12  0.01  103.06  406  100.71   
60000   205  120  119  110.04  422  107.87  2.16  0.01  103.07  405  100.68   
65000   343  130  129  110.01  420  107.80  2.20  0.01  103.06  404  100.65   
70000   290  140  139  109.97  419  107.73  2.23  0.01  103.07  404  100.63   
75000   246  150  149  109.94  417  107.68  2.25  0.01  103.11  405  100.66   
80000   308  160  159  109.91  416  107.62  2.28  0.01  103.06  403  100.59   
85000   285  170  169  109.89  415  107.57  2.30  0.01  103.09  403  100.58   
90000   282  180  179  109.87  413  107.53  2.33  0.01  103.08  401  100.55   
95000   279  190  189  109.85  412  107.49  2.35  0.01  103.17  404  100.62   
100000  313  200  199  109.83  411  107.45  2.37  0.01  103.05  400  100.49   
105000  277  210  209  109.81  411  107.42  2.39  0.01  103.22  404  100.64   
110000  321  220  219  109.79  410  107.38  2.40  0.01  103.08  400  100.50   
115000  303  230  229  109.78  409  107.35  2.42  0.01  103.09  400  100.48   
120000  328  240  239  109.76  408  107.32  2.44  0.01  103.12  400  100.48   
125000  329  250  249  109.75  407  107.29  2.45  0.01  103.11  400  100.48   
130000  265  260  259  109.73  407  107.26  2.46  0.01  103.01  397  100.38   
135000  373  270  269  109.72  406  107.23  2.48  0.01  102.94  396  100.30   
140000  295  280  279  109.71  405  107.21  2.49  0.01  102.93  395  100.28   
145000  267  290  289  109.69  405  107.18  2.50  0.01  103.04  398  100.38   
150000  290  300  299  109.68  404  107.16  2.52  0.01  103.01  397  100.35   
...     ...  ...  ...     ...  ...     ...   ...   ...     ...  ...     ...   
195000  285  390  389  109.60  400  106.98  2.60  0.01  103.21  400  100.50   
200000  271  400  399  109.59  400  106.97  2.61  0.01  103.16  398  100.44   
205000  270  410  409  109.58  399  106.95  2.62  0.01  103.16  398  100.42   
210000  222  420  419  109.58  399  106.94  2.63  0.01  103.05  395  100.28   
215000  277  430  429  109.57  399  106.92  2.63  0.01  102.93  393  100.19   
220000  150  440  439  109.56  398  106.91  2.64  0.01  102.95  393  100.17   
225000  289  450  449  109.55  398  106.89  2.65  0.01  102.95  394  100.17   
230000  213  460  459  109.55  398  106.88  2.66  0.01  102.90  392  100.13   
235000  263  470  469  109.54  397  106.86  2.66  0.01  102.90  392  100.12   
240000  254  480  479  109.53  397  106.85  2.67  0.01  102.99  395  100.21   
245000  257  490  489  109.53  397  106.84  2.67  0.01  102.91  392  100.14   
250000  252  501    0  109.52  397  106.83  2.68  0.01  102.91  392  100.11   
255000  266  511   10  109.52  396  106.82  2.69  0.01  102.87  391  100.09   
260000  204  521   20  109.51  396  106.80  2.69  0.01  102.86  391  100.07   
265000  279  531   30  109.51  396  106.79  2.70  0.01  102.91  392  100.13   
270000  196  541   40  109.50  396  106.78  2.70  0.01  102.86  391  100.07   
275000  277  551   50  109.50  395  106.77  2.71  0.01  102.90  391  100.09   
280000  277  561   60  109.49  395  106.76  2.71  0.01  102.96  392  100.

0 R: 1.000 P: 0.371 carry pockets room pocket small perfect nice hold space back
   1 R: 0.220 P: 0.068 - quality : price made padding $ leather material 'm
     11 R: 0.036 P: 0.036 ; & pro bought ... big perfectly size nice price
     12 R: 0.025 P: 0.025 inch sleeve size bought netbook hp laptops purchased amazon inches
     13 R: 0.032 P: 0.032 $ price pro quality buy cases bought reviews ... made
     14 R: 0.010 P: 0.010 years wheels handle year quality targus bags 've swiss pack
     17 R: 0.025 P: 0.025 ! gift bought loves price quality christmas amazon ... husband
     15 R: 0.026 P: 0.026 return item received amazon seller shipping back ordered service time
   2 R: 0.135 P: 0.052 protection inside sleeve ipad padding drive tablet zipper usb protect
     21 R: 0.031 P: 0.031 ! love perfectly perfect color recommend mac highly colors book
     22 R: 0.020 P: 0.020 sleeve chromebook inside perfectly snug samsung neoprene design inch nice
     23 R: 0.019 P: 0.019 pro mac retina 

ValueError: min() arg is an empty sequence